Per entrar a una sessió de Spark i iniciar un builder en DeltaLake

In [1]:
#!pip install pyspark
#!pip install delta-spark

import pyspark
from delta import *

#!wget -O "HR_comma_sep.csv" "https://mydisk.cs.upc.edu/s/3o33yciBHADiFCD/download/HR_comma_sep.csv"

builder = pyspark.sql.SparkSession.builder.appName("Shops_Deltalake") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/aina/.ivy2/cache
The jars for the packages stored in: /Users/aina/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6bd7ed64-6ea1-45f1-a7ab-3e096ea99751;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 146ms :: artifacts dl 6ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.1.0 from central in [default]
	io.delta#delta-storage;3.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   

## Comandes com si estessis a una sessió de Spark

Llegir els arxius i guardar-los en format DeltaLake, que ens garantitza que es compleixen les restriccions ACID, i dona avanatatges 




Carregar i llegir els 3 arxius de tenim

In [2]:
#Arxiu Parquet 
shops = spark.read.parquet("./datalake/shops_data/2024-04-17_shops_data.parquet")
shops.show()


+-------------------+------------------+--------------------+-------------------------+---------------------------+----------------------+--------------------+---------------------+------------------------+------------------------+-----------------------+------------------------+---------------------------+--------------------+------------------+----------------+-------------------+--------------------+-------------------+---------------+--------------------+------------------+
|attributes.objectid|attributes.osm_id2|attributes.abandoned|attributes.addr_housename|attributes.addr_housenumber|attributes.addr_street|attributes.addr_city|attributes.addr_state|attributes.addr_postcode|attributes.addr_province|attributes.addr_country|attributes.addr_district|attributes.addr_subdistrict|attributes.addr_unit|attributes.amenity|attributes.brand|attributes.building|     attributes.name|attributes.operator|attributes.shop|          geometry.x|        geometry.y|
+-------------------+-------------

In [3]:
#Exemple, nose si funciona en el nostre cas
DF = spark.sql("""SELECT salary, avg(cast(last_evaluation as float)) FROM HR GROUP BY salary""")
DF.show()

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `HR` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 56;
'Aggregate ['salary], ['salary, unresolvedalias('avg(cast('last_evaluation as float)), None)]
+- 'UnresolvedRelation [HR], [], false


In [5]:
#Guardar l'arxiu
shops.write.mode("overwrite").format("delta").save("./deltalake/shops_data/")

24/04/19 11:35:48 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [ ]:
from uszipcode import SearchEngine

# Crea un objeto SearchEngine
search = SearchEngine(simple_zipcode=True)

# Función para obtener el código postal
def get_zipcode(lat, lon):
    result = search.by_coordinates(lat=lat, lng=lon, returns=1)
    return result[0].zipcode

# Aplica la función a tus datos
df["ZipCode"] = df.apply(lambda row: get_zipcode(row["LATITUDE_X"], row["LONGITUDE_X"]), axis=1)
